# Homework 3
In this assignment, we will be building a Naïve Bayes classifier and a SVM model for the productivity satisfaction of [the given dataset](https://archive.ics.uci.edu/ml/datasets/Productivity+Prediction+of+Garment+Employees), the productivity of garment employees.

## Background
The Garment Industry is one of the key examples of the industrial globalization of this modern era. It is a highly labour-intensive industry with lots of manual processes. Satisfying the huge global demand for garment products is mostly dependent on the production and delivery performance of the employees in the garment manufacturing companies. So, it is highly desirable among the decision makers in the garments industry to track, analyse and predict the productivity performance of the working teams in their factories. 

## Dataset Attribute Information

1. **date**: Date in MM-DD-YYYY
2. **day**: Day of the Week
3. **quarter** : A portion of the month. A month was divided into four quarters
4. **department** : Associated department with the instance
5. **team_no** : Associated team number with the instance
6. **no_of_workers** : Number of workers in each team
7. **no_of_style_change** : Number of changes in the style of a particular product
8. **targeted_productivity** : Targeted productivity set by the Authority for each team for each day.
9. **smv** : Standard Minute Value, it is the allocated time for a task
10. **wip** : Work in progress. Includes the number of unfinished items for products
11. **over_time** : Represents the amount of overtime by each team in minutes
12. **incentive** : Represents the amount of financial incentive (in BDT) that enables or motivates a particular course of action.
13. **idle_time** : The amount of time when the production was interrupted due to several reasons
14. **idle_men** : The number of workers who were idle due to production interruption
15. **actual_productivity** : The actual % of productivity that was delivered by the workers. It ranges from 0-1.

### Libraries that can be used: numpy, scipy, pandas, scikit-learn, cvxpy, imbalanced-learn
Any libraries used in the discussion materials are also allowed.

#### Other Notes

 - Don't worry about not being able to achieve high accuracy, it is neither the goal nor the grading standard of this assignment. <br >
 - If not specified, you are not required to do hyperparameter tuning, but feel free to do so if you'd like.

#### Trouble Shooting
In case you have trouble installing and using imbalanced-learn(imblearn) <br >
Run the below code cell, then go to the selection bar at top: Kernel > Restart. <br >
Then try `import imblearn` to see if things work. 

In [2]:
# Install a pip package in the current Jupyter kernel
import sys
!{sys.executable} -m pip install imbalanced-learn delayed

# Exercises

## Exercise 1 - General Data Preprocessing (20 points)

Our dataset needs cleaning before building any models. Some of the cleaning tasks are common in general, but depends on what kind of models we are building, sometimes we have to do additional processing. These additional tasks will be mentioned in each of the remaining two exercises later.

Note that **we will be using this processed data from exercise 1 in each of the remaining two exercises**.

For convenience, here are the attributes that we would treat them as **categorical attributes**: `day`, `quarter`, `department`, and `team`. 

 - Drop the column `date`.
 - For each of the categorical attributes, **print out** all the unique elements.
 - For each of the categorical attributes, remap the duplicated items, if you find there are typos or spaces among the duplicated items.
     - For example, "a" and "a " should be the same, so we need to update "a " to be "a".
     - Another example, "apple" and "appel" should be the same, so you should update "appel" to be "apple".
     

 - Create another column named `satisfied` that records the productivity performance. The behavior defined as follows. **This is the dependent variable we'd like to classify in this assignment.**
     - Return True or 1 if `actual_productivity` is equal to or greater than `targeted_productivity`. Otherwise, return False or 0, which means the team fails to meet the expected performance.
 - Drop the columns `actual_productivity` and `targeted_productivity`.


 - Find and **print out** which columns/attributes that have empty vaules, e.g., NA, NaN, null, None.
 - Fill the empty values with 0.


In [3]:
import pandas as pd

# Read in data frame
df = pd.read_csv('./garments_worker_productivity.csv')

# Drop date column
df.drop('date', axis = 1, inplace = True)

# View categorical attributes
print(df.day.unique())
print(df.quarter.unique())
print(df.department.unique())
print(df.team.unique())

# Create dictionary to remap typos in 'department' column
di = {'sweing': 'sewing', 'finishing ': 'finishing'}
df.replace({'department': di}, inplace=True)

# Generate 'satisfied' column, drop productivity cols
def f(row):
    if row['actual_productivity'] >= row['targeted_productivity']:
        val = 1
    else:
        val = 0
    return val

df['satisfied'] = df.apply(f, axis=1)
df.drop('actual_productivity', axis = 1, inplace = True)
df.drop('targeted_productivity', axis = 1, inplace = True)

# Find columns with NaN's, replace with 0's
print(df.columns[df.isna().any()])
df['wip'] = df['wip'].fillna(0)

['Thursday' 'Saturday' 'Sunday' 'Monday' 'Tuesday' 'Wednesday']
['Quarter1' 'Quarter2' 'Quarter3' 'Quarter4' 'Quarter5']
['sweing' 'finishing ' 'finishing']
[ 8  1 11 12  6  7  2  3  9 10  5  4]
Index(['wip'], dtype='object')


## Exercise 2 - Naïve Bayes Classifier (40 points in total)

### Exercise 2.1 - Additional Data Preprocessing (10 points)

To build a Naïve Bayes Classifier, we need to further encode our categorical variables.

 - For each of the **categorical attribtues**, encode the set of categories to be **0 ~ (n_classes - 1)**.
     - For example, \["paris", "paris", "tokyo", "amsterdam"\] should be encoded as \[1, 1, 2, 0\].
     - Note that the order does not really matter, i.e., \[0, 0, 1, 2\] also works. But you have to start with 0 in your encodings.
     - You can find information about this encoding in the discussion materials.


 - Split the data into training and testing set with the ratio of 80:20.

In [4]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder

df2 = pd.DataFrame(OrdinalEncoder().fit_transform(df), columns = df.columns)
train, test = train_test_split(df2, test_size=0.2, random_state=21)

,quarter,department,day,team,smv,wip,over_time,incentive,idle_time,idle_men,no_of_style_change,no_of_workers,satisfied
0,0.0,1.0,3.0,7.0,41.0,328.0,101.0,37.0,0.0,0.0,0.0,57.0,1.0
1,0.0,0.0,3.0,0.0,2.0,0.0,9.0,0.0,0.0,0.0,0.0,5.0,1.0
2,0.0,1.0,3.0,10.0,9.0,247.0,44.0,20.0,0.0,0.0,0.0,25.0,1.0
3,0.0,1.0,3.0,11.0,9.0,247.0,44.0,20.0,0.0,0.0,0.0,25.0,1.0
4,0.0,1.0,3.0,5.0,40.0,367.0,20.0,20.0,0.0,0.0,0.0,51.0,1.0


### Exercise 2.2 - Naïve Bayes Classifier for Categorical Attributes (15 points)

Use the categorical attributes **only**, please build a Categorical Naïve Bayes classifier that predicts the column `satisfied`. <br >
Report the **testing result** using `classification_report`.

In [5]:
from sklearn.metrics import classification_report
from sklearn.naive_bayes import CategoricalNB

categoricals = ['day', 'quarter', 'department', 'team']
X_train, X_test = train[categoricals], test[categoricals]
y_train, y_test = train['satisfied'], test['satisfied']

NB = CategoricalNB()
NB.fit(X_train, y_train)
print(classification_report(NB.predict(X_test), y_test))

              precision    recall  f1-score   support

         0.0       0.28      0.66      0.39        29
         1.0       0.94      0.77      0.85       211

    accuracy                           0.75       240
   macro avg       0.61      0.71      0.62       240
weighted avg       0.86      0.75      0.79       240



### Exercise 2.3 - Naïve Bayes Classifier for Numerical Attributes (15 points)

Use the numerical attributes **only**, please build a Gaussian Naïve Bayes classifier that predicts the column `satisfied`. <br >
Report the **testing result** using `classification_report`.

**Remember to scale your data. The scaling method is up to you.**

In [6]:
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import StandardScaler

numericals = ['smv', 'wip', 'over_time', 'incentive', 'idle_time', 
              'idle_men', 'no_of_style_change', 'no_of_workers']
X_train, X_test = train[numericals], test[numericals] 

StandardScaler().fit_transform(X_train.values.reshape(-1,1))
StandardScaler().fit_transform(X_test.values.reshape(-1,1))
NB = GaussianNB()
NB.fit(X_train, y_train)
print(classification_report(NB.predict(X_test), y_test))

              precision    recall  f1-score   support

         0.0       0.04      1.00      0.08         3
         1.0       1.00      0.73      0.84       237

    accuracy                           0.73       240
   macro avg       0.52      0.86      0.46       240
weighted avg       0.99      0.73      0.83       240



## Exercies 3 - SVM Classifier (40 points in total)

### Exercise 3.1 - Additional Data Preprocessing (10 points)

To build a SVM Classifier, we need a different encoding for our categorical variables.

 - For each of the **categorical attribtues**, encode them with **one-hot encoding**.
     - You can find information about this encoding in the discussion materials.


 - Split the data into training and testing set with the ratio of 80:20.


In [7]:
df3 = df.copy(deep=True)
onehot_day = pd.get_dummies(df3.day)
onehot_quarter = pd.get_dummies(df3.quarter)
onehot_dept = pd.get_dummies(df3.department)
onehot_team = pd.get_dummies(df3.team)

df3.drop(['day', 'quarter', 'department', 'team'], axis=1, inplace=True)
df3 = df3.join([onehot_day, onehot_quarter, onehot_dept, onehot_team])

train, test = train_test_split(df3, test_size=0.2, random_state=21)

### Exercise 3.2 - SVM with Different Kernels (20 points)

Using all the attributes we have, please build a SVM that predicts the column `satisfied`. <br >
Specifically, please 
 - Build one SVM with **linear kernel**.
 - Build another SVM but with **rbf kernel**.
 - Report the **testing results** of **both models** using `classification report`.

The kernel is the only setting requirement. <br >
Other hyperparameter tuning is not required. But make sure they are the same in these two SVMs if you'd like to tune the model. In other words, the only difference between the two SVMs should be the kernel setting.

**Remember to scale your data. The scaling method is up to you.**

In [39]:
from sklearn.svm import LinearSVC, SVC

X_train, X_test = train.drop(columns=['satisfied']), test.drop(columns=['satisfied'])
y_train, y_test = train['satisfied'], test['satisfied']

StandardScaler().fit_transform(X_train.values.reshape(-1,1))
StandardScaler().fit_transform(X_test.values.reshape(-1,1))

svc_lin = LinearSVC(dual=False, C=10)
svc_lin.fit(X_train, y_train)

svc_rbf = SVC(kernel='rbf', random_state=1, gamma=0.03, C=7)
svc_rbf.fit(X_train, y_train)

print('Support Vector Classifier (linear kernel):')
print(classification_report(y_test, svc_lin.predict(X_test)))
print('Support Vector Classifier (radial basis function kernel):')
print(classification_report(y_test, svc_rbf.predict(X_test)))

Support Vector Classifier (linear kernel):
              precision    recall  f1-score   support

           0       1.00      0.09      0.16        68
           1       0.74      1.00      0.85       172

    accuracy                           0.74       240
   macro avg       0.87      0.54      0.50       240
weighted avg       0.81      0.74      0.65       240

Support Vector Classifier (radial basis function kernel):
              precision    recall  f1-score   support

           0       0.72      0.43      0.54        68
           1       0.81      0.94      0.87       172

    accuracy                           0.79       240
   macro avg       0.77      0.68      0.70       240
weighted avg       0.78      0.79      0.77       240



In [40]:
# from sklearn.model_selection import RandomizedSearchCV

# C = [1, 3, 5, 7, 10, 12]
# gamma = [0.03, 0.05, 0.07, 0.1, 0.15]
# p = dict(C = C, gamma = gamma)
# grid = RandomizedSearchCV(svc_rbf, p, n_jobs = -1, cv = 5)
# grid_result = grid.fit(X_train, y_train)

# print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
# means = grid_result.cv_results_['mean_test_score']
# stds = grid_result.cv_results_['std_test_score']
# params = grid_result.cv_results_['params']
# for mean, stdev, param in zip(means, stds, params):
#     print("%f (%f) with: %r" % (mean, stdev, param))

# p = dict(C = C)
# grid = RandomizedSearchCV(svc_lin, p, n_jobs = -1, cv = 5)
# grid_result = grid.fit(X_train, y_train)

# print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
# means = grid_result.cv_results_['mean_test_score']
# stds = grid_result.cv_results_['std_test_score']
# params = grid_result.cv_results_['params']
# for mean, stdev, param in zip(means, stds, params):
#     print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.795206 using {'gamma': 0.03, 'C': 3}
0.779554 (0.016156) with: {'gamma': 0.15, 'C': 3}
0.795206 (0.007312) with: {'gamma': 0.03, 'C': 3}
0.770157 (0.016896) with: {'gamma': 0.1, 'C': 7}
0.785820 (0.013910) with: {'gamma': 0.05, 'C': 7}
0.773277 (0.012542) with: {'gamma': 0.05, 'C': 10}
0.771204 (0.019681) with: {'gamma': 0.15, 'C': 5}
0.773277 (0.012985) with: {'gamma': 0.1, 'C': 5}
0.784784 (0.017409) with: {'gamma': 0.07, 'C': 5}
0.784773 (0.012002) with: {'gamma': 0.03, 'C': 12}
0.782679 (0.014618) with: {'gamma': 0.05, 'C': 1}
Best: 0.753409 using {'C': 10}
0.745048 (0.007978) with: {'C': 1}
0.745048 (0.005556) with: {'C': 3}
0.750267 (0.003444) with: {'C': 5}
0.749220 (0.006423) with: {'C': 7}
0.753409 (0.009799) with: {'C': 10}
0.748184 (0.010834) with: {'C': 12}


/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sklearn/model_selection/_search.py:285: UserWarning: The total space of parameters 6 is smaller than n_iter=10. Running 6 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


### Exercise 3.3 - SVM with Over-sampling (10 points)
 - For the column `satisfied` in our **training set**, please **print out** the frequency of each class. 
 - Oversample the **training data**. 
 - For the column `satisfied` in the oversampled data, **print out** the frequency of each class  again.
 - Re-build the 2 SVMs with the same setting you have in Exercise 3.2, but **use oversampled training data** instead.
     - Do not forget to scale the data first. As always, the scaling method is up to you.
 - Report the **testing result** with `classification_report`.

You can use ANY methods listed on [here](https://imbalanced-learn.org/stable/references/over_sampling.html#) such as RandomOverSampler or SMOTE. <br > 
You are definitely welcomed to build your own oversampler. <br >

Note that you do not have to over-sample your testing data.

In [41]:
from imblearn.over_sampling import RandomOverSampler

print('Original Training Set:')
print(y_train.value_counts())

ros = RandomOverSampler(random_state=42)
X_train_ros, y_train_ros = ros.fit_resample(X_train, y_train)

print('Oversampled Training Set:')
print(y_train_ros.value_counts(), '\n')

StandardScaler().fit_transform(X_train_ros.values.reshape(-1,1))

svc_lin = LinearSVC(dual=False, C=10)
svc_lin.fit(X_train_ros, y_train_ros)

svc_rbf = SVC(kernel='rbf', random_state=1, gamma=0.03, C=7)
svc_rbf.fit(X_train_ros, y_train_ros)

print('SVC (linear kernel, oversampled training dataset):')
print(classification_report(y_test, svc_lin.predict(X_test)))
print('SVC (radial basis function kernel, oversampled training dataset):')
print(classification_report(y_test, svc_rbf.predict(X_test)))

Original Training Set:
1    703
0    254
Name: satisfied, dtype: int64
Oversampled Training Set:
0    703
1    703
Name: satisfied, dtype: int64 

SVC (linear kernel, oversampled training dataset):
              precision    recall  f1-score   support

           0       0.47      0.74      0.57        68
           1       0.86      0.67      0.75       172

    accuracy                           0.69       240
   macro avg       0.67      0.70      0.66       240
weighted avg       0.75      0.69      0.70       240

SVC (radial basis function kernel, oversampled training dataset):
              precision    recall  f1-score   support

           0       0.68      0.56      0.61        68
           1       0.84      0.90      0.87       172

    accuracy                           0.80       240
   macro avg       0.76      0.73      0.74       240
weighted avg       0.79      0.80      0.79       240

